In [1]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os
import json

# 3rd party lib
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook
#matplotlib.rcParams.update({'font.size': 22})
#from bqplot import pyplot as plt

import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt
from gphelper import GPHelper

from importlib import reload

import warnings
warnings.filterwarnings('ignore')


from multiprocessing import Pool

Using matplotlib backend: Qt5Agg


In [10]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    if img.min() <= 0:
        img += epsilon
    return img

def get_spheroids(num=10, band='h'):
    with open('../spheroids', 'r') as f:
        spheroids= np.array(f.readlines())
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)
    
    sources = []
    for s in spheroids:
        s = s.strip()
        img = fits.getdata(f_string.format(s, band))
        segmap = fits.getdata(f_string.format(s, 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img
        del segmap
        
    return sources
        
def process_img(args):
    def denoise(img, segmap, img_id):
        noise_bank = img[segmap==0].flatten()
        other_source = np.logical_and(segmap!=0, segmap!=img_id)
        np.place(img, other_source, noise_bank)

        sep_arr = img.byteswap().newbyteorder('N')
        bkg = sep.Background(sep_arr, mask=segmap==img_id, bw=10, bh=10)
        img = img-bkg

        return img
    
    def get_rs_and_fs(img, src_map):
        cx, cy = it.img_center(img, src_map)
        xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
        rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))

        rs = rs.flatten()
        fs = img.flatten()
        sorted_rs = np.argsort(rs)

        rs = rs[sorted_rs]
        fs = fs[sorted_rs]

        return rs, fs
    
    img, segmap, img_id = args
    src_map = segmap==img_id
    img = denoise(img, segmap, img_id)
    
    rs, fs = get_rs_and_fs(img, src_map)
    
    dr = 0.05*(rs.max()-rs.min())
    fs = dt.loessc_p(rs, fs, dr, pnum=2)
    fs[fs<0]=0
    
    num_interp = 100
    _rs = np.linspace(rs.min(), rs.max(), num_interp)
    fs = np.interp(_rs, rs, fs)
    rs = _rs
    
    L = np.zeros(num_interp)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,num_interp,1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*((fs[i]+fs[i-1])/2)

    A = np.pi * rs**2
    etas = fs*A/L
    r_lim = rs<25.0
    #etas = etas[r_lim]
    mono_dec = np.argmax(np.diff(etas)>=0)
    mono_dec = np.arange(len(etas))<mono_dec
    r_lim = np.logical_and(r_lim, mono_dec)
    
    Rp = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    Fp_idx = np.square(rs - 2*Rp).argmin()

    Fp = np.cumsum(L[:Fp_idx]/L[:Fp_idx].sum())
    Ip = np.interp(Rp, rs, fs)

    return (rs, fs, Rp, Ip, etas, img_id)

In [64]:
# get imgs in other bands
spheroids_h = get_spheroids(-1, 'h')
spheroids_j = get_spheroids(-1, 'j')
spheroids_v = get_spheroids(-1, 'v')
spheroids_z = get_spheroids(-1, 'z')

In [4]:
# get the imgs in h band 
with open('h_vals.json', 'r') as f:
    processed_h = json.load(f)

In [5]:
ids = [h['id'] for h in processed_h]

# get rid of the imgs that were excluded from h
tmpj, tmpv, tmpz = [], [], []
for j, v, z, in zip(spheroids_j, spheroids_v, spheroids_z):
    if j[-1] in ids:
        tmpj.append(j)
        tmpv.append(v)
        tmpz.append(z)

spheroids_j = tmpj
spheroids_v = tmpv
spheroids_z = tmpz

In [ ]:
%%time
count = 0

processed_h, processed_j, processed_v, processed_z = [], [], [], []
for h, j, v, z in zip(spheroids_h, spheroids_j, spheroids_v, spheroids_z):
    print(count/len(spheroids_j), end='\r')
    count += 1
    
    try:
        h = process_img(h)
        j = process_img(j)
        v = process_img(v)
        z = process_img(z)
    except Exception as e:
        continue
    
    processed_h.append(h)
    processed_j.append(j)
    processed_v.append(v)
    processed_z.append(z)

In [15]:
kept_ids = [p[-1] for p in processed_j]

In [19]:
tmp_h = []
for h in processed_h:
    if h['id'] in kept_ids:
        tmp_h.append(h)
processed_h = tmp_h

In [20]:
with open('h_vals.json', 'w') as f:
    json.dump(processed_h, f)

In [21]:
for b,data in zip(['j','v','z'], [processed_j, processed_v, processed_z]):
    vals = []
    for s in data:
        v = {
            'rs' : dt._nmpy_encode(s[0]),
            'fs' : dt._nmpy_encode(s[1]),
            'rp' : s[2],
            'ip' : s[3],
            'etas' : dt._nmpy_encode(s[4]),
            'id' : s[5]
        }
        vals.append(v)

    with open(b+'_vals.json', 'w') as f:
        json.dump(vals, f)

In [24]:
tmp_h = []

for h in processed_h:
    tmp_h.append((
        dt._nmpy_decode(h['rs']),
        dt._nmpy_decode(h['fs']),
        h['rp'],
        h['ip'],
        dt._nmpy_decode(h['etas']),
        h['id']
    ))
    
processed_h = tmp_h    

In [28]:
processed_h = sorted(processed_h, key=lambda v: v[-1])
processed_j = sorted(processed_j, key=lambda v: v[-1])
processed_v = sorted(processed_v, key=lambda v: v[-1])
processed_z = sorted(processed_z, key=lambda v: v[-1])

In [63]:
for h in processed_j:
    plt.plot(h[0], h[1])

In [55]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])
j_ratios, v_ratios, z_ratios = [], [], []
h_vals, j_vals, v_vals, z_vals = [], [], [], []

for h, j, v, z in zip(processed_h, processed_j, processed_v, processed_z):
    h_rs, h_fs, h_rp, h_ip = h[0], h[1], h[3], h[4]
    j_rs, j_fs, j_rp, j_ip = j[0], j[1], j[3], j[4]
    v_rs, v_fs, v_rp, v_ip = v[0], v[1], v[3], v[4]
    z_rs, z_fs, z_rp, z_ip = z[0], z[1], z[3], z[4]
    
    # are there any values we need to pad
    if (np.sum(h_fs<=0)+np.sum(j_fs<=0)+np.sum(v_fs<=0)+np.sum(z_fs<=0)) > 0:
            epsilon = min(h_fs.min(), j_fs.min(), v_fs.min(), z_fs.min()) + 1e-3
    
            h_fs += epsilon
            j_fs += epsilon
            v_fs += epsilon
            z_fs += epsilon
    
    
    h_vals.append(h_fs)
    j_vals.append(j_fs)
    v_vals.append(v_fs)
    z_vals.append(z_fs)
    
    # normalize
    h_rs /= h_rp
    h_fs /= h_ip
    j_rs /= h_rp
    j_fs /= j_ip
    v_rs /= h_rp
    v_fs /= v_ip
    z_rs /= h_rp
    z_fs /= z_ip

    

    
    
    # get interp values
    h_fs = np.interp(interped_marks, h_rs, h_fs, left=np.nan, right=np.nan)
    j_fs = np.interp(interped_marks, j_rs, j_fs, left=np.nan, right=np.nan)
    v_fs = np.interp(interped_marks, v_rs, v_fs, left=np.nan, right=np.nan)
    z_fs = np.interp(interped_marks, z_rs, z_fs, left=np.nan, right=np.nan)
    

    j_ratios.append(j_fs/h_fs)
    v_ratios.append(v_fs/h_fs)
    z_ratios.append(z_fs/h_fs)

In [58]:
plt.figure()
plt.title('J')
for f in j_vals:
    plt.plot(f)
    
plt.figure()
plt.title('V')
for f in v_vals:
    plt.plot(interped_marks, f)
    
plt.figure()
plt.title('Z')
for f in z_vals:
    plt.plot(interped_marks, f)

ValueError: x and y must have same first dimension, but have shapes (1000,) and (100,)

In [46]:
plt.figure()
plt.title('J')
for f in j_ratios:
    plt.plot(interped_marks, f)
    
plt.figure()
plt.title('V')
for f in v_ratios:
    plt.plot(interped_marks, f)
    
plt.figure()
plt.title('Z')
for f in z_ratios:
    plt.plot(interped_marks, f)